In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import clear_output

In [ ]:
csv = pd.read_csv('Specialities_by_city.csv')
csv.head()

,City,Specialty,Link
0,ahmedabad,Dentistry(665),https://www.lybrate.com/ahmedabad/dentist
1,ahmedabad,General Physician(372),https://www.lybrate.com/ahmedabad/general-phys...
2,ahmedabad,Gynaecology(244),https://www.lybrate.com/ahmedabad/gynaecologist
3,ahmedabad,Homeopathy(193),https://www.lybrate.com/ahmedabad/homeopath
4,ahmedabad,Pediatrics(158),https://www.lybrate.com/ahmedabad/pediatrician


In [ ]:
_links = csv['Link']
_links.head()

0            https://www.lybrate.com/ahmedabad/dentist
1    https://www.lybrate.com/ahmedabad/general-phys...
2      https://www.lybrate.com/ahmedabad/gynaecologist
3          https://www.lybrate.com/ahmedabad/homeopath
4       https://www.lybrate.com/ahmedabad/pediatrician
Name: Link, dtype: object

In [ ]:
csv.iloc[0, :]

City                                         ahmedabad
Specialty                               Dentistry(665)
Link         https://www.lybrate.com/ahmedabad/dentist
Name: 0, dtype: object

In [ ]:
visited = {}
for itr in range(0, csv.shape[0], 1):
    row = csv.iloc[itr, :]
    link = row[2]
    if link in visited:
        print("Already seen")
        continue
    else:
        visited[link] = '1'
    speciality = link.split('/')[-1].replace(',', '').strip()
    csv_write = open('./Specialty/'+speciality+'.csv', 'a+')

    page_no = 1
    while (page_no < 10):
        try:
            clear_output(wait=True)
            link_new = link + '?page=' + str(page_no)
            response = requests.get(link_new)
            page_no+=1
            if response.status_code != 200:
                print("Status code: "+str(response.status_code)+' '+link)
                break
            content = response.text
            soup = BeautifulSoup(content, 'lxml')

            required_html = soup.find('div', {'class': 'grid__col-lt-20 lybMar-top-btm--half lybPad-left-right--quar'})
            grid_cells = required_html.findAll('div', {'class': 'grid'})

            try:
                for grid_cell in grid_cells:
                    print("Line no: "+str(itr)+" Page no: " + str(page_no) + ' ' +str(speciality))
                    #print(grid_cell)
                    print(grid_cell['class'])
                    if grid_cell['class'][0] != 'grid' or len(grid_cell['class']) != 1:
                        print('Skipped div')
                        continue

                    heading = grid_cell.find('h2', {'itemprop': 'name'})
                    name_link = heading.find('a')
                    name = name_link.get_text().strip()
                    profile_link = name_link['href']
                    if profile_link in visited:
                        print("Already seen")
                        continue
                    else:
                        visited[profile_link] = '1'

                    #https://www.lybrate.com/ahmedabad/doctor/dr-ganesha-m-dentist
                    city = profile_link.split('/')[3].replace(',', '').strip()

                    degree_tag = grid_cell.find('div', {'class': 'lybEllipsis ly-doctor__degree grid__col-20'})
                    degree = degree_tag.get_text().replace(',', ' ').strip()
                    degree = degree.replace('\n', '')

                    address_tag = grid_cell.find('span', {'itemprop': 'streetAddress'})
                    address = address_tag.get_text().replace(',', ' ').strip()
                    address = address.replace('\n', '')

                    right_part = grid_cell.findAll('div', {'class': 'grid__col-xs-10 grid--direction-row'})
                    rating = '0'
                    experience = '0'
                    charges = '0'
                    for i in range(0, len(right_part), 1):
                        text = right_part[i].get_text().strip()
                        if text.find('%') != -1:
                            rating = text.replace('\n', '')
                        elif text.find('experience') != -1:
                            experience = text.replace('\n', '')
                        elif text.find('₹') != -1:
                            charges = text.replace('\n', '')

                    print(name, profile_link, degree, address, rating, experience, charges, speciality, city)
                    csv_write.write(name+','+profile_link+','+degree+','+address+','+rating+','+experience+','+charges+','+speciality+','+city+'\n')
            except Exception as ex:
                print("Inner Exception: "+str(ex)+' '+link)
        except Exception as e:
            print("Exception: "+str(e)+' '+link)

    csv_write.close()

Line no: 4562 Page no: 10 veterinarian
['grid']
Already seen
Line no: 4562 Page no: 10 veterinarian
['grid', 'grid--align-self-start', 'ly-doctor__info', 'grid--justify-start']
Skipped div
Line no: 4562 Page no: 10 veterinarian
['grid', 'grid--direction-row', 'lybPad-top', 'grid__col-20']
Skipped div
Line no: 4562 Page no: 10 veterinarian
['grid']
Already seen
Line no: 4562 Page no: 10 veterinarian
['grid', 'grid--align-self-start', 'ly-doctor__info', 'grid--justify-start']
Skipped div
Line no: 4562 Page no: 10 veterinarian
['grid', 'grid--direction-row', 'lybPad-top', 'grid__col-20']
Skipped div
Line no: 4562 Page no: 10 veterinarian
['grid', 'grid__cell']
Skipped div
